# Setup

In [90]:
import contracts
import os
contracts.disable_all()
import duckietown_world as dw
from duckietown_world.svg_drawing.ipython_utils import ipython_draw_html
dw.logger.setLevel(50)

Better visualization of output

In [91]:
%%html
<style>
pre {line-height: 90%}
</style>

# Map representation

Let's load a map and see how data is represented inside:

In [93]:
print(os.path)
m = dw.load_map('ethz_amod_lab_k31')
ipython_draw_html(m);


<module 'posixpath' from '/home/linuxbrew/.linuxbrew/opt/python/lib/python3.7/posixpath.py'>


The map is a `DuckietownMap` which is an instance of `PlacedObject`. All spatially situated objects
are instances of this class.

In [94]:
type(m).mro() # see all superclasses of the object

[duckietown_world.world_duckietown.duckietown_map.DuckietownMap,
 duckietown_world.geo.placed_object.PlacedObject,
 duckietown_serialization_ds1.serialization1.Serializable,
 duckietown_serialization_ds1.serialization1.Serializable0,
 object]

## Children

The data is arranged in a hierarchy. We can visualize using the function `get_object_tree`. The hierarchy is more deep, but we clip it to 2 levels:

In [95]:
print(dw.get_object_tree(m, levels=4))

DuckietownMap

└ tilemap ┐ TileMap
          │ 
          │ ├ tile-0-5 ┐ Tile
          │ │          │
          │ │          │ └ curve_left ┐ PlacedObject
          │ │          │              │
          │ │          │              │ └ curve ┐ PlacedObject
          │ │          │              │         └───────────────
          │ │          │              └───────────────────────────
          │ │          └────────────────────────────────────────────
          │ ├ tile-1-5 ┐ Tile
          │ │          │
          │ │          │ └ straight ┐ PlacedObject
          │ │          │            │
          │ │          │            │ ├ lane1 ┐ LaneSegment
          │ │          │            │ │       └──────────────
          │ │          │            │ └ lane2 ┐ LaneSegment
          │ │          │            │         └──────────────
          │ │          │            └──────────────────────────
          │ │          └─────────────────────────────────────────
          │ ├ tile-2-5

The children are available in the `children` variable:

In [96]:
m.children

{'tilemap': TileMap(children={'tile-0-5': Tile(children={'curve_left': PlacedObject(children={'curve': PlacedObject(children={'lane1': LaneSegment(children={}, spatial_relations={}), 'lane2': LaneSegment(children={}, spatial_relations={})}, spatial_relations={'lane1': GroundTruth(() -> ('lane1',)  SE2Transform([0.0, 0.0],0.0)), 'lane2': GroundTruth(() -> ('lane2',)  SE2Transform([0.0, 0.0],1.5707963267948966))})}, spatial_relations={'curve': GroundTruth(() -> ('curve',)  SE2Transform([0.0, 0.0],4.71238898038469))})}, spatial_relations={1: GroundTruth(() -> ('curve_left',)  SE2Transform([0.0, 0.0],0.0))}), 'tile-1-5': Tile(children={'straight': PlacedObject(children={'lane1': LaneSegment(children={}, spatial_relations={}), 'lane2': LaneSegment(children={}, spatial_relations={})}, spatial_relations={'lane1': GroundTruth(() -> ('lane1',)  SE2Transform([0.0, 0.0],0.0)), 'lane2': GroundTruth(() -> ('lane2',)  SE2Transform([0.0, 0.0],3.141592653589793))})}, spatial_relations={1: GroundTruth(

In [63]:
m.children['tilemap'].children

{'tile-0-4': Tile(children={'curve_left': PlacedObject(children={'curve': PlacedObject(children={'lane1': LaneSegment(children={}, spatial_relations={}), 'lane2': LaneSegment(children={}, spatial_relations={})}, spatial_relations={'lane1': GroundTruth(() -> ('lane1',)  SE2Transform([0.0, 0.0],0.0)), 'lane2': GroundTruth(() -> ('lane2',)  SE2Transform([0.0, 0.0],1.5707963267948966))})}, spatial_relations={'curve': GroundTruth(() -> ('curve',)  SE2Transform([0.0, 0.0],4.71238898038469))})}, spatial_relations={1: GroundTruth(() -> ('curve_left',)  SE2Transform([0.0, 0.0],0.0))}),
 'tile-1-4': Tile(children={'straight': PlacedObject(children={'lane1': LaneSegment(children={}, spatial_relations={}), 'lane2': LaneSegment(children={}, spatial_relations={})}, spatial_relations={'lane1': GroundTruth(() -> ('lane1',)  SE2Transform([0.0, 0.0],0.0)), 'lane2': GroundTruth(() -> ('lane2',)  SE2Transform([0.0, 0.0],3.141592653589793))})}, spatial_relations={1: GroundTruth(() -> ('straight',)  SE2Tran

You can use the notation below to get a child in a compact way:

In [97]:
lane_segment = m['tilemap/tile-0-1/straight/lane1']

KeyError: 'Cannot find child straight in []'

In this case we can see how a Tile has a child `curve_left` with a child `curve` with two children `lane1` and `lane2`:

In [104]:
tile = m.children['tilemap'].children['tile-4-0']
print(dw.get_object_tree(tile, attributes=False, levels=10))

Tile

└ straight ┐ PlacedObject
           │ 
           │ ├ lane1 ┐ LaneSegment
           │ │       └──────────────
           │ └ lane2 ┐ LaneSegment
           │         └──────────────
           └──────────────────────────


In [105]:
print(dw.get_object_tree(tile, attributes=True, levels=10))

Tile

 kind: straight
 drivable: true
 

└ straight ┐ PlacedObject
           │ 
           │ ├ lane1 ┐ LaneSegment
           │ │       │
           │ │       │  width: 0.376
           │ │       │  control_points:
           │ │       │  -   ~SE2Transform:
           │ │       │          p:
           │ │       │          - -0.5
           │ │       │          - -0.22
           │ │       │  -   ~SE2Transform:
           │ │       │          p:
           │ │       │          - 0.5
           │ │       │          - -0.22
           │ │       │
           │ │       └──────────────────────
           │ └ lane2 ┐ LaneSegment
           │         │
           │         │  width: 0.376
           │         │  control_points:
           │         │  -   ~SE2Transform:
           │         │          p:
           │         │          - -0.5
           │         │          - -0.22
           │         │  -   ~SE2Transform:
           │         │          p:
           │         │          -

In [115]:
lane1 = tile['straight/lane2']
ipython_draw_html(lane1);


In [118]:
curve = tile['straight/lane1']
print(dw.get_object_tree(curve, attributes=True, spatial_relations=True, levels=10))

LaneSegment

 width: 0.376
 control_points:
 -   ~SE2Transform:
         p:
         - -0.5
         - -0.22
 -   ~SE2Transform:
         p:
         - 0.5
         - -0.22
 


In [119]:
ipython_draw_html(curve);

In [125]:
lane = tile['straight/lane2']._copy()

ipython_draw_html(lane);

In [126]:
lane.width

0.376

In [127]:
lane.control_points

[SE2Transform([-0.5, -0.22],0.0), SE2Transform([0.5, -0.22],0.0)]

## Parametrization of lanes

The lane is parametrized with a parameter `beta` that interpolates among the control points.

Here we create an animation of the center point for different `beta`s.

In [128]:
import numpy as np
npoints = len(lane.control_points)
betas = list(np.linspace(-1, npoints + 1, 20))
print(betas)

[-1.0, -0.7894736842105263, -0.5789473684210527, -0.368421052631579, -0.1578947368421053, 0.05263157894736836, 0.26315789473684204, 0.4736842105263157, 0.6842105263157894, 0.894736842105263, 1.1052631578947367, 1.3157894736842106, 1.526315789473684, 1.7368421052631575, 1.9473684210526314, 2.1578947368421053, 2.3684210526315788, 2.578947368421052, 2.789473684210526, 3.0]


In [129]:
transforms = []
for beta in betas:
    # call the function `center_point` to get the center point (in SE(2))
    p = lane.center_point(beta)
    transform = dw.SE2Transform.from_SE2(p)
    transforms.append(transform)

ground_truth = dw.SampledSequence(betas, transforms)
lane.set_object('traveling-point', dw.PlacedObject(), ground_truth=ground_truth)
ipython_draw_html(lane);

Scrub the timeline to see the animation.

In [130]:
import geometry as geo
q = geo.SE2_from_translation_angle([+0.05, -0.05], np.deg2rad(20))

In [131]:
lane.set_object('db18-4', dw.DB18(), ground_truth=dw.SE2Transform.from_SE2(q))
lane_pose = lane.lane_pose_from_SE2(q)

lane.set_object('marker3', dw.PlacedObject(), ground_truth=lane_pose.center_point)

In [132]:
lane.children

{'traveling-point': PlacedObject(children={}, spatial_relations={}),
 'db18-4': DB18(children={}, spatial_relations={}),
 'marker3': PlacedObject(children={}, spatial_relations={})}

In [133]:
lane.spatial_relations

{0: GroundTruth(() -> ('traveling-point',)  SampledSequence(timestamps=[-1.0, -0.7894736842105263, -0.5789473684210527, -0.368421052631579, -0.1578947368421053, 0.05263157894736836, 0.26315789473684204, 0.4736842105263157, 0.6842105263157894, 0.894736842105263, 1.1052631578947367, 1.3157894736842106, 1.526315789473684, 1.7368421052631575, 1.9473684210526314, 2.1578947368421053, 2.3684210526315788, 2.578947368421052, 2.789473684210526, 3.0], values=[SE2Transform([-0.6, -0.2199999988079071],0.0), SE2Transform([-0.5789473684210527, -0.2199999988079071],0.0), SE2Transform([-0.5578947368421052, -0.2199999988079071],0.0), SE2Transform([-0.5368421052631579, -0.2199999988079071],0.0), SE2Transform([-0.5157894736842106, -0.2199999988079071],0.0), SE2Transform([-0.44736842105263164, -0.2199999988079071],0.0), SE2Transform([-0.23684210526315796, -0.2199999988079071],0.0), SE2Transform([-0.026315789473684292, -0.2199999988079071],0.0), SE2Transform([0.18421052631578938, -0.2199999988079071],0.0), 

In [134]:
ipython_draw_html(lane);

In [81]:
assert lane_pose.correct_direction

In [82]:
assert lane_pose.inside

In [83]:
assert lane